In [ ]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy
print(numpy.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

print(torch.__version__)
print(torch.version.cuda)


In [ ]:
class PostcardDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.transform = transform

        # Filter
        self.df = self.df.dropna(subset=["akon_id", "country_id", "latitude", "longitude"])

        self.countries = sorted(self.df["country_id"].unique())
        self.country_to_idx = {c: i for i, c in enumerate(self.countries)}
        self.idx_to_country = {i: c for c, i in self.country_to_idx.items()}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row["akon_id"] + ".jpg")
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        country_idx = self.country_to_idx[row["country_id"]]
        lat = float(row["latitude"])
        lon = float(row["longitude"])

        # Normierte Koordinaten
        coords = torch.tensor([lat / 90.0, lon / 180.0], dtype=torch.float32)

        return image, country_idx, coords

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

csv_path = "akon_postcards_public_domain.csv"
image_dir = "images/256"
dataset = PostcardDataset(csv_path, image_dir, transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)


# Beispielbild anzeigen
img, label_idx, coords = dataset[0]
plt.imshow(img.permute(1, 2, 0) * 0.5 + 0.5)
plt.title(f"Label-Index: {label_idx}")
plt.show()



In [ ]:
embedding_dim = 48
z_dim = 100
coord_dim = 2
num_countries = len(dataset.country_to_idx)

country_embedding = nn.Embedding(num_countries, embedding_dim).to(device)



In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim + embedding_dim + coord_dim, 1024 * 4 * 4),
            nn.ReLU(True)
        )

        self.conv = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),

            nn.ConvTranspose2d(32, 3, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, z, country_embed, coords):
        x = torch.cat([z, country_embed, coords], dim=1)
        x = self.fc(x).view(-1, 1024, 4, 4)
        return self.conv(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.label_proj = nn.Linear(embedding_dim + coord_dim, 256 * 256)

        self.model = nn.Sequential(
            nn.Conv2d(4, 32, 4, 2, 1),
            nn.LeakyReLU(0.2),

            nn.Conv2d(32, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 1024, 4, 2, 1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2),

            nn.Flatten(),
            nn.Linear(1024 * 4 * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, img, country_embed, coords):
        label = torch.cat([country_embed, coords], dim=1)
        label_map = self.label_proj(label).view(-1, 1, 256, 256)
        x = torch.cat([img, label_map], dim=1)
        return self.model(x)


In [ ]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)

g_opt = torch.optim.Adam(generator.parameters(), lr=2e-4)
d_opt = torch.optim.Adam(discriminator.parameters(), lr=2e-4)

criterion = nn.BCELoss()

In [ ]:
import time
from torchvision.utils import save_image

fixed_noise = torch.randn(16, z_dim).to(device)
fixed_country = torch.randint(0, num_countries, (16,)).to(device)
fixed_coords = torch.rand(16, 2).to(device)


In [ ]:
import os
import time
import torch
from torchvision.utils import save_image

# Ordner vorbereiten
os.makedirs("training/DCGANv2/epoche_bilder", exist_ok=True)
os.makedirs("training/DCGANv2/epoche_schritte", exist_ok=True)

# Feste Inputs für gleichbleibende Samples
fixed_noise = torch.randn(16, z_dim).to(device)
fixed_country = torch.randint(0, num_countries, (16,)).to(device)
fixed_coords = torch.randn(16, coord_dim).to(device)  

# Checkpoint Setup
start_epoch = 0
checkpoint_path = "training/DCGANv2/epoche_schritte/checkpoint_epoch_latest.pth"

if os.path.exists(checkpoint_path):
    print(" Checkpoint gefunden, lade...")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    generator.load_state_dict(checkpoint["generator_state_dict"])
    discriminator.load_state_dict(checkpoint["discriminator_state_dict"])
    g_opt.load_state_dict(checkpoint["g_opt_state_dict"])
    d_opt.load_state_dict(checkpoint["d_opt_state_dict"])
    start_epoch = checkpoint["epoch"]
    print(f" Weiter bei Epoche {start_epoch}")
else:
    print(" Kein Checkpoint gefunden, starte bei Epoche 0.")

num_epochs = 10000

for epoch in range(start_epoch, num_epochs):
    start_time = time.time()
    print(f"\n Starte Epoche {epoch+1}/{num_epochs}")

    for imgs, country_idxs, coords in dataloader:
        imgs = imgs.to(device)
        country_idxs = country_idxs.to(device)
        coords = coords.to(device)
        batch_size = imgs.size(0)

        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # --- Discriminator ---
        z = torch.randn(batch_size, z_dim).to(device)
        country_embed_disc = country_embedding(country_idxs)  # neues embedding für Disc
        fake_imgs = generator(z, country_embed_disc, coords)

        real_preds = discriminator(imgs, country_embed_disc, coords)
        fake_preds = discriminator(fake_imgs.detach(), country_embed_disc, coords)

        d_loss = criterion(real_preds, real_labels) + criterion(fake_preds, fake_labels)

        d_opt.zero_grad()
        d_loss.backward()
        d_opt.step()

        # --- Generator ---
        z = torch.randn(batch_size, z_dim).to(device)
        country_embed_gen = country_embedding(country_idxs)  # neues embedding für Gen
        fake_imgs = generator(z, country_embed_gen, coords)
        fake_preds = discriminator(fake_imgs, country_embed_gen, coords)
        g_loss = criterion(fake_preds, real_labels)

        g_opt.zero_grad()
        g_loss.backward()
        g_opt.step()

    # Epoch Zusammenfassung
    duration = time.time() - start_time
    print(f"Epoche {epoch+1} abgeschlossen | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f} | ⏱️ {duration:.2f}s")

    # Alle 10 Epochen Beispielbilder speichern
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            embed = country_embedding(fixed_country)
            samples = generator(fixed_noise, embed, fixed_coords)
            save_image(samples * 0.5 + 0.5,
                       f"training/DCGANv2/epoche_bilder/gen_samples_epoch_{epoch+1}.png",
                       nrow=4)

    # Alle 10 Epochen Checkpoint speichern
    if (epoch + 1) % 10 == 0:
        checkpoint_data = {
            'epoch': epoch + 1,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'g_opt_state_dict': g_opt.state_dict(),
            'd_opt_state_dict': d_opt.state_dict(),
        }

        torch.save(checkpoint_data, f"training/DCGANv2/epoche_schritte/checkpoint_epoch_{epoch+1}.pth")
        torch.save(checkpoint_data, checkpoint_path)  # aktuellsten Stand überschreiben


In [ ]:

torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")

z = torch.randn(1, z_dim)

In [ ]:
"""
o = torch.ones_like(z)
o[:,0] = 5
z+=o
z = z.to(device)
"""

In [ ]:
# Generiere Bild für ein bestimmtes Land und Koordinaten (z.B. Köln)
z = torch.randn(1, z_dim).to(device)

# Beispiel: normierte Koordinaten von Köln (Latitude / 90, Longitude / 180)
# Köln ca. 50.94°N, 6.96°E
lat_norm = 50.94 / 90.0
lon_norm = 6.96 / 180.0
coords = torch.tensor([[lat_norm, lon_norm]], dtype=torch.float32).to(device)

# Country-ID (z.B. Deutschland "DE")
city_country = "DE"
country_idx = torch.tensor([dataset.country_to_idx[city_country]]).to(device)

# Hole das Country-Embedding
country_embed = country_embedding(country_idx)

# Modell in Eval-Modus
generator.eval()
with torch.no_grad():
    gen_img = generator(z, country_embed, coords).squeeze().cpu()
    plt.imshow(gen_img.permute(1, 2, 0) * 0.5 + 0.5)
    plt.title(f"Generierte Postkarte: {city_country}")
    plt.axis("off")
    plt.show()


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd

embedding_dim = 50  # Beispielgröße für Embedding

# Länderliste und Mapping erstellen
all_countries = sorted(dataset.df["country_id"].unique())
country_to_idx = {c: i for i, c in enumerate(all_countries)}

# Country-Embedding Layer (neu initialisiert)
country_embedding = torch.nn.Embedding(len(all_countries), embedding_dim).to(device)

# Indizes für alle Länder holen
all_idxs = torch.tensor([country_to_idx[c] for c in all_countries]).to(device)

# Embeddings holen
embeddings = country_embedding(all_idxs).detach().cpu().numpy()

# t-SNE
tsne = TSNE(n_components=2, random_state=42)
emb_2d = tsne.fit_transform(embeddings)

# DataFrame zum Plotten bauen
df_tsne = pd.DataFrame({
    "country": all_countries,
    "x": emb_2d[:, 0],
    "y": emb_2d[:, 1]
})

# Plot t-SNE
plt.figure(figsize=(12, 10))
plt.scatter(df_tsne["x"], df_tsne["y"])

for i, country in enumerate(df_tsne["country"]):
    plt.annotate(country, (df_tsne.loc[i, "x"], df_tsne.loc[i, "y"]), fontsize=8)

plt.title("Country Embeddings via t-SNE")
plt.show()

# PCA
pca = PCA(n_components=2)
emb_2d_pca = pca.fit_transform(embeddings)

df_pca = pd.DataFrame({
    "country": all_countries,
    "x": emb_2d_pca[:, 0],
    "y": emb_2d_pca[:, 1]
})

plt.figure(figsize=(12, 10))
for country in df_pca["country"].unique():
    subset = df_pca[df_pca["country"] == country]
    plt.scatter(subset["x"], subset["y"], label=country, s=10)

plt.title("Country Embeddings via PCA")
plt.legend(title="country_id", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize="small")
plt.tight_layout()
plt.show()
